# Estatística Aplicada II
## Primeira Lista de Exercícios
### Estudante: Clístenes Grizafis Bento

<p>Com a base de dados “imoveiscwbav” obter os seguintes resultados com o auxílio do “R”</p>
<p>Estimar três modelos (Ridge, Lasso e Elasticnet) para explicar a variável Y (price), as demais variáveis da base de dados são todas variáveis explicativas; particione a base de dados em 80% para treino e 20% para teste; e apresente os resultados:</p>

   - O valor ótimo do lambda para os modelos;
   - O valor do alpha para o modelo ElasticNet;
   - Os valores dos parâmetros para os modelos;
   - O R^2 e RMSE dos modelos estimados;
   - Apresente os resultados de uma predição proposta por você mesmo para os modelos (valor estimado e intervalos de confiança).

##### Importando as bibliotecas necessárias e dados

In [8]:
library(plyr)
library(readr)
library(dplyr)
library(caret)
library(ggplot2)
library(repr)
library(glmnet)

load("./Arquivos_para_R/imoveiscwbav.RData")
dataset <- imoveiscwbav
glimpse(dataset)
gc

Observations: 541
Variables: 20
$ price  <dbl> 1100000, 895000, 2513600, 755000, 1099000, 475000, 463900, 1...
$ age    <dbl> 15, 11, 2, 25, 1, 31, 2, 1, 11, 1, 3, 3, 3, 11, 3, 20, 3, 3,...
$ parea  <dbl> 150, 165, 146, 163, 107, 96, 75, 122, 63, 97, 92, 138, 199, ...
$ tarea  <dbl> 190, 210, 275, 238, 189, 124, 90, 227, 87, 180, 130, 253, 40...
$ bath   <dbl> 4, 4, 4, 3, 3, 2, 2, 3, 2, 2, 2, 3, 5, 3, 2, 3, 5, 5, 2, 4, ...
$ ensuit <dbl> 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 3, 3, 1, 3, ...
$ garag  <dbl> 2, 2, 3, 2, 2, 1, 1, 2, 1, 2, 3, 2, 3, 2, 2, 1, 2, 3, 2, 2, ...
$ plaz   <dbl> 0.08058169, 0.16635098, 0.05607530, 0.32159391, 0.14663511, ...
$ park   <dbl> 0.7132806, 0.6983694, 1.3129824, 2.1099578, 1.0175299, 1.970...
$ trans  <dbl> 2.3862709, 2.2463043, 2.6314112, 2.1387003, 1.7978931, 0.994...
$ kidca  <dbl> 1.4109813, 1.8625914, 1.5914926, 1.6215857, 1.2572430, 1.097...
$ school <dbl> 0.9028108, 0.9355790, 0.4517910, 0.4478709, 0.8841994, 0.391...
$ health <dbl> 0.414

function (verbose = getOption("verbose"), reset = FALSE, full = TRUE) 
{
    res <- .Internal(gc(verbose, reset, full))
    res <- matrix(res, 2L, 7L, dimnames = list(c("Ncells", "Vcells"), 
        c("used", "(Mb)", "gc trigger", "(Mb)", "limit (Mb)", 
            "max used", "(Mb)")))
    if (all(is.na(res[, 5L]))) 
        res[, -5L]
    else res
}
<bytecode: 0x0000000040ee5260>
<environment: namespace:base>

##### Separando os dados de treino e os de teste

In [9]:
# criando semente pseudo aleatória para verificação futura
set.seed(1)

indices <- createDataPartition(dataset$price, p=0.8, list=F)
treino <- dataset[indices,]
teste <- dataset[-indices,]

##### Alterando escala das variaveis

In [19]:
cols <- c('price', 'age', 'parea', 'tarea', 'bath',
         'ensuit', 'garag', 'plaz','park', 'trans',
         'kidca', 'school', 'health', 'bike')

pre_proc_val <- preProcess(treino[,cols], method = c("center", "scale"))

treino[,cols] <- predict(pre_proc_val, treino[,cols])
teste[,cols] <- predict(pre_proc_val, teste[,cols])

print("valores de treino")
summary(treino)
print("valores de teste")
summary(teste)

[1] "valores de treino"


     price              age              parea              tarea          
 Min.   :-1.3512   Min.   :-1.0471   Min.   :-2.17312   Min.   :-1.872522  
 1st Qu.:-0.7324   1st Qu.:-0.8890   1st Qu.:-0.83396   1st Qu.:-0.882164  
 Median :-0.1389   Median :-0.3354   Median :-0.04264   Median : 0.005744  
 Mean   : 0.0000   Mean   : 0.0000   Mean   : 0.00000   Mean   : 0.000000  
 3rd Qu.: 0.4155   3rd Qu.: 0.7716   3rd Qu.: 0.74108   3rd Qu.: 0.782663  
 Max.   : 6.1479   Max.   : 2.9857   Max.   : 2.39220   Max.   : 3.762277  
      bath             ensuit            garag              plaz        
 Min.   :-2.5996   Min.   :-1.5890   Min.   :-2.7684   Min.   :-1.6640  
 1st Qu.:-0.8758   1st Qu.:-0.4978   1st Qu.:-1.2590   1st Qu.:-0.8550  
 Median :-0.0139   Median :-0.4978   Median : 0.2504   Median :-0.1820  
 Mean   : 0.0000   Mean   : 0.0000   Mean   : 0.0000   Mean   : 0.0000  
 3rd Qu.: 0.8480   3rd Qu.: 0.5934   3rd Qu.: 0.2504   3rd Qu.: 0.7331  
 Max.   : 2.5718   Max.   : 1.

[1] "valores de teste"


     price              age               parea              tarea         
 Min.   :-1.2572   Min.   :-1.04711   Min.   :-1.68615   Min.   :-1.68470  
 1st Qu.:-0.7310   1st Qu.:-0.88896   1st Qu.:-0.84918   1st Qu.:-0.80533  
 Median :-0.1577   Median :-0.49358   Median :-0.04264   Median :-0.07963  
 Mean   :-0.0544   Mean   :-0.08712   Mean   :-0.03041   Mean   :-0.06399  
 3rd Qu.: 0.3500   3rd Qu.: 0.73209   3rd Qu.: 0.67260   3rd Qu.: 0.64606  
 Max.   : 2.3357   Max.   : 2.27406   Max.   : 1.90523   Max.   : 1.91816  
      bath             ensuit              garag               plaz         
 Min.   :-2.5996   Min.   :-1.588981   Min.   :-2.76839   Min.   :-1.67190  
 1st Qu.:-0.8758   1st Qu.:-0.497814   1st Qu.:-1.25899   1st Qu.:-0.77106  
 Median :-0.0139   Median :-0.497814   Median : 0.25041   Median :-0.02648  
 Mean   :-0.1508   Mean   :-0.008318   Mean   : 0.08113   Mean   : 0.07582  
 3rd Qu.: 0.8480   3rd Qu.: 0.593354   3rd Qu.: 0.25041   3rd Qu.: 0.92688  
 Max. 

#### Função que calcula e retorna $R^2$ e RMSE

In [50]:
eval_results <- function(true, predicted, df) {
  SSE <- sum((predicted - true)^2)
  SST <- sum((true - mean(true))^2)
  R_square <- 1 - SSE / SST
  RMSE = sqrt(SSE/nrow(df))
  
  # Model performance metrics
  data.frame(
    RMSE = RMSE,
    Rsquare = R_square
  )
}

##### Regressão Ridge

In [21]:
cols_reg <- c('price', 'age', 'parea', 'tarea', 'bath',
         'ensuit', 'garag', 'plaz','park', 'trans',
         'kidca', 'school', 'health', 'bike', 'barb',
         'balc', 'elev', 'fitg', 'party', 'categ')

dummies <- dummyVars(price ~ age+parea+tarea+bath+ 
                     ensuit+garag+plaz+park+trans+kidca+
                     school+health+bike+barb+balc+elev+fitg+
                     party+categ, 
                     data = dataset[,cols_reg])

train_dummies <- predict(dummies, newdata = treino[,cols_reg])

test_dummies <- predict(dummies, newdata = teste[,cols_reg])

print(dim(train_dummies)); print(dim(test_dummies))

[1] 434  19
[1] 107  19


In [22]:
# Dados para o modelo
x = as.matrix(train_dummies)
y_train = treino$price

x_test = as.matrix(test_dummies)
y_test = teste$price

In [28]:
#Identficando o valor ótimo de lambda
lambdas <- 10^seq(2, -3, by = -.1)
ridge_lamb <- cv.glmnet(x, y_train, alpha = 0, 
                      lambda = lambdas)
best_lambda_ridge <- ridge_lamb$lambda.min
best_lambda_ridge

[1] 0.1

##### O melhor lambda obtido é 0.1

In [29]:
ridge_reg = glmnet(x, y_train, nlambda = 25, alpha = 0, 
                   family = 'gaussian', 
                   lambda = best_lambda_ridge)

summary(ridge_reg)

          Length Class     Mode   
a0         1     -none-    numeric
beta      19     dgCMatrix S4     
df         1     -none-    numeric
dim        2     -none-    numeric
lambda     1     -none-    numeric
dev.ratio  1     -none-    numeric
nulldev    1     -none-    numeric
npasses    1     -none-    numeric
jerr       1     -none-    numeric
offset     1     -none-    logical
call       7     -none-    call   
nobs       1     -none-    numeric

In [48]:
# obtendo valores dos parâmetros
ridge_reg[["beta"]]

19 x 1 sparse Matrix of class "dgCMatrix"
                s0
age    -0.17994688
parea   0.15907495
tarea   0.21631677
bath    0.04261451
ensuit  0.18954877
garag   0.20506605
plaz    0.04714112
park   -0.05375536
trans   0.03345027
kidca   0.01621329
school -0.00131847
health -0.01054514
bike   -0.04790725
barb   -0.06878716
balc    0.15377909
elev   -0.18946965
fitg    0.23383601
party   0.06497685
categ   0.46258571

In [54]:
#predição dos dados de Teste
predictions_test <- predict(ridge_reg, s = best_lambda_ridge, 
                            newx = x_test)
eval_results(y_test, predictions_test, test)

RMSE,Rsquare
0.1792301,0.8487347


**Avaliação:** O $R^2$ está próximo de 1, mas não tão próximo, o que é um bom sinal pois significa que tem menos chances de o modelo estar sofrendo overfitting. O RMSE está próximo de zero, o que é bom sinal, significa que poucos erros foram cometidos.

##### Realizando predição através de variáveis obtidas através da médiana de todo banco de dados

In [69]:
# O valor esperado é R$ 880000 
price <- (median(dataset$price)-pre_proc_val[["mean"]][["price"]])/pre_proc_val[["std"]][["price"]]
age <- (median(dataset$age)-pre_proc_val[["mean"]][["age"]])/pre_proc_val[["std"]][["age"]]
parea <- (median(dataset$parea)-pre_proc_val[["mean"]][["parea"]])/pre_proc_val[["std"]][["parea"]]
tarea <- (median(dataset$tarea)-pre_proc_val[["mean"]][["tarea"]])/pre_proc_val[["std"]][["tarea"]]
bath <- (median(dataset$bath)-pre_proc_val[["mean"]][["bath"]])/pre_proc_val[["std"]][["bath"]]
ensuit <- (median(dataset$ensuit)-pre_proc_val[["mean"]][["ensuit"]])/pre_proc_val[["std"]][["ensuit"]]
garag <- (median(dataset$garag)-pre_proc_val[["mean"]][["garag"]])/pre_proc_val[["std"]][["garag"]]
plaz <- (median(dataset$plaz)-pre_proc_val[["mean"]][["plaz"]])/pre_proc_val[["std"]][["plaz"]]
park <- (median(dataset$park)-pre_proc_val[["mean"]][["park"]])/pre_proc_val[["std"]][["park"]]
trans <- (median(dataset$trans)-pre_proc_val[["mean"]][["trans"]])/pre_proc_val[["std"]][["trans"]]
kidca <- (median(dataset$kidca)-pre_proc_val[["mean"]][["kidca"]])/pre_proc_val[["std"]][["kidca"]]
school <- (median(dataset$school)-pre_proc_val[["mean"]][["school"]])/pre_proc_val[["std"]][["school"]]
health <- (median(dataset$health)-pre_proc_val[["mean"]][["health"]])/pre_proc_val[["std"]][["health"]]
bike <- (median(dataset$bike)-pre_proc_val[["mean"]][["bike"]])/pre_proc_val[["std"]][["bike"]]
barb <- 0
balc <- 0
elev <- 0
fitg <- 0
party <- 0
categ <- 0

# Constuirndo matriz com dados para predição
our_pred <- as.matrix(data.frame(age=age,
                                parea=parea,
                                tarea=tarea,
                                bath=bath,
                                ensuit=ensuit,
                                garag=garag,
                                plaz=plaz,
                                park=park,
                                trans=trans,
                                kidca=kidca,
                                school=school,
                                health=health,
                                bike=bike,
                                barb=barb,
                                balc=balc,
                                elev=elev,
                                fitg=fitg,
                                party=party,
                                categ=categ))

In [70]:
# Fazendo a predição
predict_our_ridge <- predict(ridge_reg, s = best_lambda_ridge, 
                     newx = our_pred)
predict_our_ridge

1
57.237


In [72]:
# Intervalo de confiança para nosso Exemplo
n <- nrow(treino)
m <- predict_our_ridge
s <- pre_proc_val[["std"]][["price"]]
dam <- s/sqrt(n)
CIlwr_ridge <- m + (qnorm(0.025))*dam
CIupr_ridge <- m - (qnorm(0.025))*dam 

CIlwr_ridge
CIupr_ridge

1
57.14292


1
57.33108


**Avaliação:** O valor obtido assim como o intervalo de confiança tiveram grande divergência com o valor esperado.

##### Regressão Lasso

In [73]:
# Escolhendo  melhor lambda como parâmetro

lambdas <- 10^seq(2, -3, by = -.1)

# Setting alpha = 1 implements lasso regression
lasso_lamb <- cv.glmnet(x, y_train, alpha = 1, 
                       lambda = lambdas, 
                       standardize = TRUE, nfolds = 5)

# Best 
best_lambda_lasso <- lasso_lamb$lambda.min 
best_lambda_lasso

lasso_model <- glmnet(x, y_train, alpha = 1, 
                      lambda = best_lambda_lasso, 
                      standardize = TRUE)

[1] 0.007943282

##### O melhor lambda obtido é 0.00794328234724281

In [74]:
# Visualizando os parâmetros calculados
lasso_model[["beta"]]

19 x 1 sparse Matrix of class "dgCMatrix"
                 s0
age    -0.177787541
parea   0.160244253
tarea   0.239394524
bath    0.005550922
ensuit  0.215324160
garag   0.214458904
plaz    0.043167047
park   -0.056688502
trans   0.030263567
kidca   0.008624390
school  .          
health -0.002502411
bike   -0.038298740
barb   -0.066286730
balc    0.147531615
elev   -0.179246307
fitg    0.240380064
party   0.043609304
categ   0.490774990

In [76]:
# Fazendo as predições a avaliando o modelo lasso na base teste
predictions_test <- predict(lasso_model, s = best_lambda_lasso, 
                            newx = x_test)
eval_results(y_test, predictions_test, teste)

RMSE,Rsquare
0.3313211,0.8459347


**Avaliação:** O $R^2$ está próximo de 1, mas não tão próximo, o que é um bom sinal pois significa que tem menos chances de o modelo estar sofrendo overfitting. O RMSE está menos próximo de zero do que o modelo de regressão Ridge.

##### Realizando predição através de variáveis obtidas através da médiana de todo banco de dados

In [79]:
# O valor esperado é R$ 880000 
predict_our_lasso <- predict(lasso_model, s = best_lambda_lasso, 
                          newx = our_pred)
predict_our_lasso

1
61.51794


In [81]:
# Intervalo de confiança para nosso Exemplo
n <- nrow(treino)
m <- predict_our_lasso
s <- pre_proc_val[["std"]][["price"]]
dam <- s/sqrt(n)
CIlwr_lasso <- m + (qnorm(0.025))*dam
CIupr_lasso <- m - (qnorm(0.025))*dam 

CIlwr_lasso
CIupr_lasso

1
61.42386


1
61.61202


**Avaliação:** O valor obtido assim como o intervalo de confiança tiveram grande divergência com o valor esperado, mas o resultado foi melhor que o do modelo Ridge.

##### Regressão ElasticNet

In [82]:
# Ajustando controle de treino
train_cont <- trainControl(method = "repeatedcv",
                           number = 10,
                           repeats = 5,
                           search = "random",
                           verboseIter = TRUE)

In [84]:
# treinando o modelo
price ~ age+parea+tarea+bath+ 
                     ensuit+garag+plaz+park+trans+kidca+
                     school+health+bike+barb+balc+elev+fitg+
                     party+categ

In [85]:
# melhor ajuste de parâmetro
elastic_reg$bestTune

,alpha,lambda
10,0.8963074,0.006091447


##### O melhor alfa obtido foi 0.8963074, e lambda 0.006091447

In [87]:
elastic_reg[["finalModel"]][["beta"]]

   [[ suppressing 69 column names 's0', 's1', 's2' ... ]]


19 x 69 sparse Matrix of class "dgCMatrix"
                                                                          
age    . .          .          .          .          .          .         
parea  . .          .          .          .          .          .         
tarea  . 0.06221618 0.09989076 0.12772340 0.14929387 0.16906064 0.18712835
bath   . .          .          .          .          .          .         
ensuit . .          .          0.01504467 0.04071750 0.06428228 0.08589871
garag  . .          0.03222980 0.05994032 0.08179667 0.10176534 0.12002760
plaz   . .          .          .          .          .          .         
park   . .          .          .          .          .          .         
trans  . .          .          .          .          .          .         
kidca  . .          .          .          .          .          .         
school . .          .          .          .          .          .         
health . .          .          .          .          .   

##### Fazendo as predições nas bases de teste e avaliando o modelo

In [88]:
predictions_test <- predict(elastic_reg, x_test)
eval_results(y_test, predictions_test, teste)

RMSE,Rsquare
0.3326286,0.8447163


**Avaliação:** O $R^2$ está próximo de 1, mas não tão próximo, o que é um bom sinal pois significa que tem menos chances de o modelo estar sofrendo overfitting. O RMSE está menos próximo de zero do que o modelo de regressão Ridge.

##### Realizando predição através de variáveis obtidas através da médiana de todo banco de dados

In [91]:
predict_our_elastic <- predict(elastic_reg,our_pred)
predict_our_elastic

[1] 61.32302

In [92]:
# Intervalo de confiança para nosso Exemplo
n <- nrow(treino)
m <- predict_our_elastic
s <- pre_proc_val[["std"]][["price"]]
dam <- s/sqrt(n)
CIlwr_elastic <- m + (qnorm(0.025))*dam
CIupr_elastic <- m - (qnorm(0.025))*dam 

CIlwr_elastic
CIupr_elastic

[1] 61.22894

[1] 61.4171

**Avaliação:** O valor obtido assim como o intervalo de confiança tiveram grande divergência com o valor esperado, mas o resultado foi melhor que o do modelo Ridge.

### **Avaliação Final:** Dentre os modelos testados o que obteve melhor resultado foi o lasso. Todavia em todos modelos o valor obtido assim como o intervalo de confiança tiveram grande divergência com o valor esperado.